[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/crewai/crewai-query-agent-as-tool.ipynb)

## Weaviate Query Agent with Crew AI

This notebook will show you how to define the Weaviate Query Agent as a tool through the Crew AI.

## Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
2. Install Crew AI with `pip install crewai`
3. Install the Weaviate Agents package with `pip install weaviate-agents`
4. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](https://github.com/weaviate/recipes/blob/main/integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


In [ ]:
!pip install --quiet weaviate-client weaviate-agents crewai crewai-tools ipywidgets

## Import libraries and keys

In [2]:
import weaviate
from weaviate_agents.query import QueryAgent
import os

from crewai.tools import tool
from crewai import Agent
from crewai import Task
from crewai import Crew, Process
from pydantic import BaseModel, Field
from typing import Type
from crewai.tools import BaseTool


<frozen importlib._bootstrap>:1342: ImportWarning: Cannot set an attribute on 'crewai.rag' for child module 'embeddings'
<frozen importlib._bootstrap>:1342: ImportWarning: Cannot set an attribute on 'crewai.rag' for child module 'storage'


In [3]:
#os.environ["WEAVIATE_URL"] = ""
#os.environ["WEAVIATE_API_KEY"] = ""

## Define Weaviate Query Agent as a tool

In [4]:
class WeaviateQuerySchema(BaseModel):
    """Input for WeaviateQueryAgentTool."""

    query: str = Field(
        ...,
        description="The query to search retrieve relevant information from the Weaviate database. Pass only the query, not the question.",
    )


class WeaviateQueryAgentTool(BaseTool):
    name: str = Field(default="Weaviate Query Agent")
    description: str = Field(
        default="Send a query to the database and get the response."
    )
    args_schema: Type[BaseModel] = WeaviateQuerySchema

    def send_query_agent_request(self, query: str) -> str:
        """
        Send a query to the database and get the response.

        Args:
            query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

        Returns:
            str: The response from the database containing relevant information.
        """

        weaviate_client = weaviate.connect_to_weaviate_cloud(
            cluster_url=os.getenv("WEAVIATE_URL"),
            auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        )
        query_agent = QueryAgent(
            client=weaviate_client,
            collections=[
                "WeaviateBlogChunk" # we are using the Weaviate Embeddings for our Blogs collection
            ],
        )
        runner = query_agent.run(query)
        print("runner", runner)
        return runner.final_answer

    def _run(self, query: str) -> str:
        return self.send_query_agent_request(query)
    

query_agent_tool = WeaviateQueryAgentTool()

In [5]:
researcher = Agent(
    role="Blog Content Researcher",
    goal="Find relevant blog posts and extract key information",
    backstory="You're specialized in analyzing blog content to extract insights and answers",
    verbose=True,
    tools=[query_agent_tool],
    llm='gpt-4'
)

In [6]:
research_task = Task(
    description="Research blog posts about packaging software applications with Docker",
    expected_output="A summary of key information from relevant blog posts",
    agent=researcher
)

In [7]:
blog_crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    process=Process.sequential,
    verbose=True,
    tracing=True
)

## Query Time

In [ ]:
result = blog_crew.kickoff()

print(result)

<frozen importlib._bootstrap_external>:781: ResourceWarning: unclosed database in <sqlite3.Connection object at 0x1189e76a0>
<frozen importlib._bootstrap_external>:781: ResourceWarning: unclosed database in <sqlite3.Connection object at 0x11975ed40>


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Blog Content Researcher                                                                                 │
│                                                                                                                 │
│  Task: Research blog posts about packaging software applications with Docker                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5ccb9ad5-eb97-4281-b3fa-6543ba32c1f6                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/var/folders/4j/js2lp8b17zs2cvk9v9pl26cc0000gn/T/ipykernel_29783/1787021759.py:38: DeprecationWarning: 
QueryAgent.run() is deprecated and will be removed in a future release. Use QueryAgent.ask() instead.
  runner = query_agent.run(query)

...

/Users/dudanogueira/dev/weaviate/recipes/integrations/llm-agent-frameworks/crewai/.venv/lib/python3.13/site-packages/rich/text.py:748: ResourceWarning: unclosed database in <sqlite3.Connection object at 0x11975ed40>
  spans.sort(key=itemgetter(0, 1))


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Blog Content Researcher                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  There are several insightful blog posts and tutorials related to packaging software applications using         │
│  Docker, especially in the context of deploying and running Weaviate, a vector search engine, and related       │
│  applications:                                                                                                  │
│                                                                                                                 │
│  1. **Getting Started with Weaviate via Docker**: You can start development quickly by running a simple Docker  │
│  command or using a `docker-compose.yml` file to customize your instance. Weaviate's Docker images are          │
│  available and come with improvements such as faster imports, updates, and deletions, which enhance upload      │
│  times for frequent changes. Some posts highlight simplified Docker configurations where Weaviate runs without  │
│  need for complex setup.                                                                                        │
│                                                                                                                 │
│  2. **Enabling Modules in Docker Compose**: When deploying Weaviate locally with Docker, you often enable       │
│  specific modules like `text2vec-openai`, `generative-openai`, or `sum-transformers` by specifying environment  │
│  variables in `docker-compose.yml`. This allows rich features such as vectorization and generation using        │
│  OpenAI or other models. API keys like OpenAI's can be passed either in environment variables or during         │
│  runtime.                                                                                                       │
│                                                                                                                 │
│  3. **Building Full-Stack Applications Using Docker**: Tutorials guide you through building full-stack apps,    │
│  for example, an image search app with Python, Flask, and Weaviate. These tutorials assume familiarity with     │
│  spinning up Weaviate instances via Docker and vector search concepts, showcasing how Docker streamlines local  │
│  development and deployment.                                                                                    │
│                                                                                                                 │
│  4. **Using Docker with AI Agents like Auto-GPT**: Docker helps package and run complex AI agents, such as      │
│  Auto-GPT, ensuring the containers can communicate (e.g., with Weaviate). These approaches include Docker or    │
│  virtual environments, with `.env` files for configuration. Docker commands to build and run such containers    │
│  link them to memory backends like Weaviate.                                                                    │
│                                                                                                                 │
│  5. **Backups and Persistence Using Docker Volumes**: Docker Compose files are tailored to include volumes and  │
│  environment variables that enable local backups and persistence for Weaviate data, thereby enhancing           │
│  reliability and data safety in containerized environments.                                                     │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e09d9c02-ee99-481f-8e54-f175a375fe5e                                                                     │
│  Agent: Blog Content Researcher                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5ccb9ad5-eb97-4281-b3fa-6543ba32c1f6                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: There are several insightful blog posts and tutorials related to packaging software              │
│  applications using Docker, especially in the context of deploying and running Weaviate, a vector search        │
│  engine, and related applications:                                                                              │
│                                                                                                                 │
│  1. **Getting Started with Weaviate via Docker**: You can start development quickly by running a simple Docker  │
│  command or using a `docker-compose.yml` file to customize your instance. Weaviate's Docker images are          │
│  available and come with improvements such as faster imports, updates, and deletions, which enhance upload      │
│  times for frequent changes. Some posts highlight simplified Docker configurations where Weaviate runs without  │
│  need for complex setup.                                                                                        │
│                                                                                                                 │
│  2. **Enabling Modules in Docker Compose**: When deploying Weaviate locally with Docker, you often enable       │
│  specific modules like `text2vec-openai`, `generative-openai`, or `sum-transformers` by specifying environment  │
│  variables in `docker-compose.yml`. This allows rich features such as vectorization and generation using        │
│  OpenAI or other models. API keys like OpenAI's can be passed either in environment variables or during         │
│  runtime.                                                                                                       │
│                                                                                                                 │
│  3. **Building Full-Stack Applications Using Docker**: Tutorials guide you through building full-stack apps,    │
│  for example, an image search app with Python, Flask, and Weaviate. These tutorials assume familiarity with     │
│  spinning up Weaviate instances via Docker and vector search concepts, showcasing how Docker streamlines local  │
│  development and deployment.                                                                                    │
│                                                                                                                 │
│  4. **Using Docker with AI Agents like Auto-GPT**: Docker helps package and run complex AI agents, such as      │
│  Auto-GPT, ensuring the containers can communicate (e.g., with Weaviate). These approaches include Docker or    │
│  virtual environments, with `.env` files for configuration. Docker commands to build and run such containers    │
│  link them to memory backends like Weaviate.                                                                    │
│                                                                                                                 │
│  5. **Backups and Persistence Using Docker Volumes**: Docker Compose files are tailored to include volumes and  │
│  environment variables that enable local backups and persistence for Weaviate data, thereby enhancing           │
│  reliability and data safety in containerized environm

╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ bd19de78-b2ec-4f10-bbf7-d0d86b0f5f01                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/bd19de78-b2ec-4f1 │
│ 0-bbf7-d0d86b0f5f01?access_code=TRACE-2f63187f43                             │
│ 🔑 Access Code: TRACE-2f63187f43                                             │
╰──────────────────────────────────────────────────────────────────────────────╯
